## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saint George,US,37.1041,-113.5841,72.00,24,1,4.61,clear sky
1,1,Turukhansk,RU,65.8167,87.9833,-6.72,100,100,2.06,overcast clouds
2,2,Carutapera,BR,-1.1950,-46.0200,77.72,88,88,6.78,overcast clouds
3,3,Faanui,PF,-16.4833,-151.7500,82.62,73,26,22.62,scattered clouds
4,4,Rikitea,PF,-23.1203,-134.9692,79.41,70,4,15.14,clear sky


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip "))
max_temp = float(input("What is the maximum temperature you would like for your trip "))

What is the minimum temperature you would like for your trip75
What is the maximum temperature you would like for your trip90


In [15]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Carutapera,BR,-1.1950,-46.0200,77.72,88,88,6.78,overcast clouds
3,3,Faanui,PF,-16.4833,-151.7500,82.62,73,26,22.62,scattered clouds
4,4,Rikitea,PF,-23.1203,-134.9692,79.41,70,4,15.14,clear sky
6,6,Georgetown,MY,5.4112,100.3354,80.01,94,20,2.30,light rain
9,9,Tondano,ID,1.3038,124.9112,77.00,94,40,1.14,moderate rain
...,...,...,...,...,...,...,...,...,...,...
700,700,Jevargi,IN,17.0167,76.7667,85.80,20,28,7.38,scattered clouds
702,702,Porto De Moz,BR,-1.7483,-52.2383,81.93,75,98,2.15,overcast clouds
703,703,San Gabriel,US,34.0961,-118.1058,84.99,13,1,5.91,clear sky
704,704,Luwuk,ID,-0.9516,122.7875,77.22,90,100,4.99,light rain


In [16]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City ID                230
City                   230
Country                230
Lat                    230
Lng                    230
Max Temp               230
Humidity               230
Cloudiness             230
Wind Speed             230
Current Description    230
dtype: int64

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = clean_df.dropna()

In [18]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Carutapera,BR,77.72,overcast clouds,-1.1950,-46.0200,
3,Faanui,PF,82.62,scattered clouds,-16.4833,-151.7500,
4,Rikitea,PF,79.41,clear sky,-23.1203,-134.9692,
6,Georgetown,MY,80.01,light rain,5.4112,100.3354,
9,Tondano,ID,77.00,moderate rain,1.3038,124.9112,
21,Samastipur,IN,75.13,clear sky,25.8500,85.7833,
26,Aquiraz,BR,86.00,few clouds,-3.9014,-38.3911,
27,Sayla,IN,80.29,scattered clouds,22.5500,71.4667,
28,Atuona,PF,81.05,overcast clouds,-9.8000,-139.0333,
39,Kahului,US,78.80,scattered clouds,20.8947,-156.4700,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        pass

In [26]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Carutapera,BR,77.72,overcast clouds,-1.1950,-46.0200,Lidera
3,Faanui,PF,82.62,scattered clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
4,Rikitea,PF,79.41,clear sky,-23.1203,-134.9692,Pension Maro'i
6,Georgetown,MY,80.01,light rain,5.4112,100.3354,Cititel Penang
9,Tondano,ID,77.00,moderate rain,1.3038,124.9112,Villa Bella
...,...,...,...,...,...,...,...
700,Jevargi,IN,85.80,scattered clouds,17.0167,76.7667,Apna Ghar
702,Porto De Moz,BR,81.93,overcast clouds,-1.7483,-52.2383,Dormitório labelle
703,San Gabriel,US,84.99,clear sky,34.0961,-118.1058,Hilton Los Angeles/San Gabriel
704,Luwuk,ID,77.22,light rain,-0.9516,122.7875,Hotel Santika Luwuk


In [27]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [30]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30,30), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))